In [45]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word. And always plus 100 to the result"""
    return len(word) + 100

my_tools = [get_word_length]

In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from tools.simple_tools import my_tools
from tools.tool_execution import execute_tool_calls

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o-mini",  # 모델명
)

llm_with_tools = llm.bind_tools(my_tools)

chain = llm_with_tools | JsonOutputToolsParser(tools=my_tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word 'teddynote'?")
# 질의내용
question = "What is the length of the word '대한민국의 수도는 어디인가요?'"

# 질의
chain = llm_with_tools | JsonOutputToolsParser(tools=my_tools) | execute_tool_calls

aa = chain.invoke(question)
print(aa)

NameError: name 'my_tools' is not defined

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [get_word_length]

# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)


# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "`teddynote` 는 몇글자야?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'teddynote'}`


109`teddynote`는 109글자입니다.

> Finished chain.
`teddynote`는 109글자입니다.
